In [8]:
import os
import cv2
import math
import glob
import pandas as pd
import shutil
import random
import itertools

import numpy as np
import pandas as pd


from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont

from Roots_point_detect_tool import *
from pattern_detection_tool import*

from tqdm.notebook import tqdm as tqdm

In [2]:
Match_result_csv_path = os.path.join('./csv_file/PBL_match_result_all_cej_bone_dilation1_replace_area_rm_300_dropna.csv')
Match_result_csv = pd.read_csv(Match_result_csv_path)
Match_result_csv['PBL_stage'] = pd.to_numeric(Match_result_csv['PBL_stage'],errors='coerce')
Match_result_csv['PBL_stage'] = Match_result_csv['PBL_stage'].fillna(-999).astype(int)
# Match_result_csv.to_csv('./lawrence_compare.csv')

In [3]:
os_sys = '/root/notebooks/'

In [4]:
json_file_list = glob.glob('/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/json_file/*.json')
print(len(json_file_list))
dest_folder = './4_result_with_label'

for json_path in tqdm(json_file_list):
    src_files = json_path
    dest_path = os.path.join(dest_folder, json_path.split('/')[-1])
    #shutil.copyfile(src_files, dest_path)

2973


In [9]:
import os
import shutil

def remove_check_point(path):
    for file_name in os.listdir(path):
        #print(file_name)
        if 'checkpoints' in file_name:
            shutil.rmtree(os.path.join(path, file_name))
        if '__pycache__' in file_name:
            shutil.rmtree(os.path.join(path, file_name))

In [10]:
parent_folder_path = './'
remove_check_point(parent_folder_path)

for folder_name in tqdm(sorted(os.listdir(parent_folder_path)), total = len(os.listdir(parent_folder_path))):
    if os.path.isdir(os.path.join(parent_folder_path, folder_name)):
        remove_check_point(os.path.join(parent_folder_path, folder_name))
        print('Find {} files in [{}] folder'.format(len(os.listdir(os.path.join(parent_folder_path, folder_name))),folder_name))

Find 12 files in [.git] folder
Find 2511 files in [1_o_image] folder
Find 8972 files in [1_roots_mask] folder
Find 2511 files in [1_seg_image] folder
Find 8969 files in [2_seg_img_with_cej] folder
Find 0 files in [3_draw_all_points_on_o_img] folder
Find 0 files in [3_draw_all_points_on_seg_img] folder
Find 746 files in [3_pattern_detection_result] folder
Find 0 files in [4_result_with_label] folder
Find 25 files in [Result_comparison] folder
Find 3 files in [Show_case] folder
Find 41 files in [csv_file] folder



#### In 1_ is 
- 1.Original image.
- 2.Unet find root image.
- 3.Unet 3 class segmentation.

#### In 2_ is 
- 1.Split all color block in root image to image_name_x.
- 2.So one image would become many images.

#### In 3_ 826-469=357 
- 1.If 2_ non_zero pixel < 5000 or abs(slope(moment of inertia)) < 1.5 would filter out.
- 2.Filled up the remaining image by it's moment of inertia scale by it's hight and width.
- 3.Using segmentaiton tooth class (green pixel) to be mask to modify the over area filled up tooth.

#### In 4_ paste the 2_ image in segmentation image  
- 1.If 2_ non_zero pixel < 5000 or abs(slope(moment of inertia)) < 1.5 would filter out.
- 2.The remaining root would be paste on segmentation image

#### In 5_ using 

In [7]:
Match_result_csv_path = os.path.join(parent_folder_path,'csv_file/PBL_match_result_all_cej_bone_dilation1_replace_area_rm_300_dropna.csv')
Match_result_csv = pd.read_csv(Match_result_csv_path)


In [8]:
# def write_info_on_image(image_path,tabel,save_path,count):
#     image_name = get_name_from_path(image_path,True)
#     image_name_False = get_name_from_path(image_path,False)
#     #print(image_name)
#     image_shape = cv2.imread(image_path).shape
#     image = Image.open(image_path)
#     draw = ImageDraw.Draw(image)
#     if image_shape[1]<600:
#         font = ImageFont.truetype('Roboto-Bold.ttf', size=25)
#     else:
#         font = ImageFont.truetype('Roboto-Bold.ttf', size=35)
#     (x, y) = (20, 10)
#     error_ = False
#     error_value = None
#     if not tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name))].empty:
#         tooth_name = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name))]['PBL_predict_tooth'].values[0].split('_')[0]
#         #print(tooth_name)
#         try:
#             LPBL = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_L'.format(tooth_name)]))]['PBL_value'].values[0]
#             L_predict_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_L'.format(tooth_name)]))]['PBL_stage'].values[0]
#             L_Label_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_L'.format(tooth_name)]))]['state'].values[0]
#         except:
#             LPBL = -888
#             L_predict_stage = -888
#             L_Label_stage = -888
#         try:
#             RPBL = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_R'.format(tooth_name)]))]['PBL_value'].values[0]
#             R_predict_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_R'.format(tooth_name)]))]['PBL_stage'].values[0]
#             R_Label_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_R'.format(tooth_name)]))]['state'].values[0]
#         except:
#             RPBL = -888
#             R_predict_stage = -888
#             R_Label_stage = -888
#         #print(L_predict_stage,R_predict_stage,L_Label_stage,R_Label_stage)
#         if L_predict_stage != L_Label_stage or R_predict_stage != R_Label_stage:
#             error_ = True
#             if L_predict_stage == -999 and L_Label_stage != -999:
#                 error_value = 11##human can find model can't
#             elif R_predict_stage == -999 and R_Label_stage != -999:
#                 error_value = 12##human can find model can't
#             elif L_predict_stage == -999 and L_Label_stage != -999 and R_predict_stage == -999 and R_Label_stage != -999:
#                 error_value = 1##human can find model can't
#             elif L_predict_stage != -999 and L_Label_stage == -999:
#                 error_value = 21##model can find human can't
#             elif R_predict_stage != -999 and R_Label_stage == -999:
#                 error_value = 22##model can find human can't
#             elif L_predict_stage != -999 and L_Label_stage == -999 and R_predict_stage != -999 and R_Label_stage == -999:
#                 error_value = 2##model can find human can't
#             elif L_predict_stage == -888 and L_Label_stage == -888:
#                 error_value = 31##one PBL = -888
#             elif R_predict_stage == -888 and R_Label_stage == -888:
#                 error_value = 32##one PBL = -888
#             elif L_predict_stage == -888 and L_Label_stage == -888 and R_predict_stage == -888 and R_Label_stage == -888:
#                 error_value = 3##one PBL = -888
#             elif L_predict_stage != -999 and L_predict_stage != -888 and L_Label_stage != -999 and L_Label_stage != -888:
#                 error_value = 0##error_value            
#             elif R_predict_stage != -999 and R_predict_stage != -888 and R_Label_stage != -999 and R_Label_stage != -888:
#                 error_value = 0##error_value            
#             elif L_predict_stage != -999 and L_predict_stage != -888 and L_Label_stage != -999 and L_Label_stage != -888 and R_predict_stage != -999 and R_predict_stage != -888 and R_Label_stage != -999 and R_Label_stage != -888:
#                 error_value = 0##error_value  
#             else:
#                 print('class error!!!')
#                 print(L_predict_stage,L_Label_stage,R_predict_stage,R_Label_stage)
#         message = "LPBL:{}% P_S:{} L_S:{}\nRPBL:{}% P_S:{} L_S:{}".format(round(float(LPBL),2),int(L_predict_stage),int(L_Label_stage),round(float(RPBL),2),int(R_predict_stage),int(R_Label_stage))
#         color = 'rgb(255, 0, 0)'
#         draw.text((x, y), message, fill=color, font=font)
#     else:
#         count+=1
#         pass
#     if error_:
#         #print(error_value)
#         if error_value ==0:
#             image.save(os.path.join(save_path,image_name_False+'_err00.PNG'))
#         elif error_value ==1:
#             image.save(os.path.join(save_path,image_name_False+'_err10.PNG'))
#         elif error_value ==11:
#             image.save(os.path.join(save_path,image_name_False+'_err11.PNG'))
#         elif error_value ==12:
#             image.save(os.path.join(save_path,image_name_False+'_err12.PNG'))
#         elif error_value ==2:
#             image.save(os.path.join(save_path,image_name_False+'_err20.PNG'))
#         elif error_value ==21:
#             image.save(os.path.join(save_path,image_name_False+'_err21.PNG'))
#         elif error_value ==22:
#             image.save(os.path.join(save_path,image_name_False+'_err22.PNG'))
#         elif error_value ==3:
#             image.save(os.path.join(save_path,image_name_False+'_err30.PNG'))
#         elif error_value ==31:
#             image.save(os.path.join(save_path,image_name_False+'_err31.PNG'))
#         elif error_value ==32:
#             image.save(os.path.join(save_path,image_name_False+'_err32.PNG'))
#         else:
#             print('error_value error')
#     else:
#         image.save(os.path.join(save_path,image_name_False+'.PNG'))
#     return count

In [9]:
def write_info_on_image(image_path,tabel,save_path,count):
    image_name = get_name_from_path(image_path,True)
    #print(image_name)
    image_name_False = get_name_from_path(image_path,False)
    #print(image_name)
    image_shape = cv2.imread(image_path).shape
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    image_left = Image.open(image_path)
    draw_left = ImageDraw.Draw(image_left)
    image_right = Image.open(image_path)
    draw_right = ImageDraw.Draw(image_right)
    if image_shape[1]<600:
        font = ImageFont.truetype('Roboto-Bold.ttf', size=25)
    else:
        font = ImageFont.truetype('Roboto-Bold.ttf', size=35)
    (x, y) = (20, 10)
    error_ = False
    error_value = None
    if not tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name))].empty:
        tooth_name = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name))]['PBL_predict_tooth'].values[0].split('_')[0]
        #print(tooth_name)
        try:
            LPBL = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_L'.format(tooth_name)]))]['PBL_value'].values[0]
            L_predict_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_L'.format(tooth_name)]))]['PBL_stage'].values[0]
            L_Label_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_L'.format(tooth_name)]))]['stage'].values[0]
            #print(L_predict_stage,L_Label_stage)
            LPBL = float(LPBL)
            L_predict_stage = float(L_predict_stage)
            L_Label_stage = float(L_Label_stage)
            #print('Left side get')
        except:
            #print('Left side No_match')
            LPBL = -888
            L_predict_stage = -888
            L_Label_stage = -888
        try:
            RPBL = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_R'.format(tooth_name)]))]['PBL_value'].values[0]
            R_predict_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_R'.format(tooth_name)]))]['PBL_stage'].values[0]
            R_Label_stage = tabel.loc[(tabel['tooth_path'] == '{}'.format(image_name)) & (tabel['PBL_predict_tooth'].isin(['{}_R'.format(tooth_name)]))]['stage'].values[0]
            #print(R_predict_stage,R_Label_stage)
            RPBL = float(RPBL)
            R_predict_stage = float(R_predict_stage)
            R_Label_stage = float(R_Label_stage)
            #print('Rright side get')
        except:
            #print('Rright side No_match')
            RPBL = -888
            R_predict_stage = -888
            R_Label_stage = -888
        #print(L_predict_stage,L_Label_stage,R_predict_stage,R_Label_stage,error_value)
        color = 'rgb(255, 0, 0)'
        
        if(L_predict_stage != -999 and L_Label_stage != -999 and L_predict_stage != -888 and L_Label_stage != -888):  
            message = "tooth:{}\nLPBL:{}% P_S:{} L_S:{}".format(tooth_name,round(float(LPBL),2),int(L_predict_stage),int(L_Label_stage))
            draw_left.text((x, y), message, fill=color, font=font)
            image_left.save(os.path.join(save_path,image_name_False+'kkk_Relabel_Left.PNG'))
        if(R_predict_stage != -999 and R_Label_stage != -999 and R_predict_stage != -888 and R_Label_stage != -888):  
            message = "tooth:{}\nRPBL:{}% P_S:{} L_S:{}".format(tooth_name,round(float(RPBL),2),int(R_predict_stage),int(R_Label_stage))
            draw_right.text((x, y), message, fill=color, font=font)
            image_right.save(os.path.join(save_path,image_name_False+'kkk_Relabel_Right.PNG'))
            
        message = "tooth:{}\nLPBL:{}% P_S:{} L_S:{}\nRPBL:{}% P_S:{} L_S:{}".format(tooth_name,round(float(LPBL),2),int(L_predict_stage),int(L_Label_stage),round(float(RPBL),2),int(R_predict_stage),int(R_Label_stage))
        draw.text((x, y), message, fill=color, font=font)
        if (L_predict_stage != L_Label_stage) or (R_predict_stage != R_Label_stage):
            error_value = 0
            if (L_predict_stage == -999 and L_Label_stage != -999) or (R_predict_stage == -999 and R_Label_stage != -999):
                error_value = 1##human can find model can't
                #print('err1')
                image.save(os.path.join(save_path,image_name_False+'kkk_cannot_model.PNG'))
                
            if (L_predict_stage != -999 and L_Label_stage == -999) or (R_predict_stage != -999 and R_Label_stage == -999):
                error_value = 1##model can find human can't
                #print('err2')
                image.save(os.path.join(save_path,image_name_False+'kkk_cannot_human.PNG'))
                
            if (L_predict_stage == -888 and L_Label_stage == -888) or (R_predict_stage == -888 and R_Label_stage == -888):
                error_value = 1##have loss side of tooth
                #print('err3')
                image.save(os.path.join(save_path,image_name_False+'kkk_loss_side.PNG'))
                
            if ((int(L_predict_stage) >= 0) and (int(L_Label_stage) >= 0)):
                if (int(L_predict_stage) > int(L_Label_stage)):
                    L_d = max(0,L_predict_stage)-max(0,L_Label_stage)
                    if os.path.isfile(os.path.join(save_path,image_name_False+'kkk_diff_over_predict_{}.PNG'.format(int(L_d)))):
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_over_predict_r{}.PNG'.format(int(L_d))))
                    else:
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_over_predict_{}.PNG'.format(int(L_d))))
                if (int(L_predict_stage) < int(L_Label_stage)):
                    L_d = max(0,L_Label_stage)-max(0,L_predict_stage)
                    if os.path.isfile(os.path.join(save_path,image_name_False+'kkk_diff_under_predict_{}.PNG'.format(int(L_d)))):
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_under_predict_r{}.PNG'.format(int(L_d))))
                    else:
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_under_predict_{}.PNG'.format(int(L_d))))
                if (int(L_predict_stage) == int(L_Label_stage)):
                    if os.path.isfile(os.path.join(save_path,image_name_False+'kkk_R_side_diff_L_side_same.PNG')):
                        image.save(os.path.join(save_path,image_name_False+'kkk_R_side_diff_L_side_samer.PNG'))
                    else:
                        image.save(os.path.join(save_path,image_name_False+'kkk_R_side_diff_L_side_same.PNG'))
                        
            if ((int(R_predict_stage) >= 0) and (int(R_Label_stage) >= 0)):
                if (int(R_predict_stage) > int(R_Label_stage)):
                    R_d = max(0,R_predict_stage)-max(0,R_Label_stage)
                    if os.path.isfile(os.path.join(save_path,image_name_False+'kkk_diff_over_predict_{}.PNG'.format(int(R_d)))):
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_over_predict_r{}.PNG'.format(int(R_d))))
                    else:
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_over_predict_{}.PNG'.format(int(R_d))))
                if (int(R_predict_stage) < int(R_Label_stage)):
                    R_d = max(0,R_Label_stage)-max(0,R_predict_stage)
                    if os.path.isfile(os.path.join(save_path,image_name_False+'kkk_diff_under_predict_{}.PNG'.format(int(R_d)))):
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_under_predict_r{}.PNG'.format(int(R_d))))
                    else:
                        image.save(os.path.join(save_path,image_name_False+'kkk_diff_under_predict_{}.PNG'.format(int(R_d))))
                if (int(R_predict_stage) == int(R_Label_stage)):
                    if os.path.isfile(os.path.join(save_path,image_name_False+'kkk_L_side_diff_R_side_same.PNG')):
                        image.save(os.path.join(save_path,image_name_False+'kkk_L_side_diff_R_side_samer.PNG'))
                    else:
                        image.save(os.path.join(save_path,image_name_False+'kkk_L_side_diff_R_side_same.PNG'))
        else:
            image.save(os.path.join(save_path,image_name_False+'kkk_same_judg.PNG'))
    else:
        message = "no any tooth match(json false)"
        color = 'rgb(0, 0, 255)'
        draw.text((x, y), message, fill=color, font=font)
        image.save(os.path.join(save_path,image_name_False+'kkk_match_f.PNG'))
    return count

In [10]:
#Write info of all detect result on image
path = os.path.join(parent_folder_path,'3_pattern_detection_result/*.PNG')
image_path_list = glob.glob(path)
image_path_list.sort()

save_path = os.path.join(parent_folder_path,'4_result_with_label')

count = 0

for index,image_path in tqdm(enumerate(image_path_list),total=len(image_path_list)):
#     if 'NN_180829_111336_16DC2D' not in image_path:
#         continue
    count = write_info_on_image(image_path,Match_result_csv,save_path,count)
print(count)


0


In [11]:
count = 84
print(count)

84


In [12]:
#Show the difference between the prediction and the doctor's judgment
o_img_folder = os.path.join(parent_folder_path,'1_o_image')
seg_img_folder = os.path.join(parent_folder_path,'1_seg_image')

path = os.path.join(parent_folder_path,'4_result_with_label')
image_path_list = glob.glob(path+'/*.PNG')
image_path_list.sort()

print(len(image_path_list))
count = 0
for index,image_path in tqdm(enumerate(image_path_list),total=len(image_path_list)):
    image_name = get_name_from_path(image_path,False)

    try:
        value = image_name[-1]
        value = int(value)
    except:
        value = 0
        
    if value == 2 :
#         print(image_path)
#         o_image_name = image_name.split('kkk')[0][:-2]
#         print('Predict got different value')
#         o_image_path = os.path.join(o_img_folder,o_image_name+'.PNG')
#         seg_image_path = os.path.join(seg_img_folder,o_image_name+'.PNG')
#         plt.figure(figsize=(15,15))
#         plt.subplot(131)
#         plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
#         plt.subplot(132)
#         plt.imshow(cv2.imread(o_image_path,0),cmap='gray')
#         plt.subplot(133)
#         plt.imshow(cv2.cvtColor(cv2.imread(seg_image_path), cv2.COLOR_BGR2RGB))
#         plt.show()
        count +=1
    else:
        pass
print(count)

19240



237


In [13]:
# #Show the difference between the prediction and the doctor's judgment
# o_img_folder = os.path.join(parent_folder_path,'1_o_image')
# seg_img_folder = os.path.join(parent_folder_path,'1_seg_image')

# path = os.path.join(parent_folder_path,'4_result_with_label')
# image_path_list = glob.glob(path+'/*.PNG')
# image_path_list.sort()

# print(len(image_path_list))
# count = 0
# for index,image_path in tqdm(enumerate(image_path_list),total=len(image_path_list)):
#     image_name = get_name_from_path(image_path,False)

#     try:
#         value = image_name[-1]
#         value = int(value)
#     except:
#         value = 0
        
#     #if value == 2 and count <200:
#     if value == 2:
#         print(image_path)
#         o_image_name = image_name.split('kkk')[0][:-2]
#         print('Predict got different value')
#         o_image_path = os.path.join(o_img_folder,o_image_name+'.PNG')
#         seg_image_path = os.path.join(seg_img_folder,o_image_name+'.PNG')
#         plt.figure(figsize=(15,15))
#         plt.subplot(131)
#         plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
#         plt.subplot(132)
#         plt.imshow(cv2.imread(o_image_path,0),cmap='gray')
#         plt.subplot(133)
#         plt.imshow(cv2.cvtColor(cv2.imread(seg_image_path), cv2.COLOR_BGR2RGB))
#         plt.show()
#         count +=1
#     else:
#         pass
# print(count)

In [14]:
root_folder_path = "./PBL_detection/4_result_with_label/*.PNG"
root_img_path_list = glob.glob(root_folder_path)
print(len(root_img_path_list))

0


In [15]:
root_img = cv2.imread(root_img_path_list[0],0)
print(root_img.shape)
print(show_pixel_set(root_img))

ret,thresh = cv2.threshold(root_img,127,255,0)
contours_all,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

point_x = []
point_y = []
for contours in contours_all:
    if(len(contours))==2:
        continue
    idx_list = []
    
    for idx in range(len(contours)):
        im = cv2.cvtColor(root_img,cv2.COLOR_GRAY2BGR)
        point = contours[idx][0]
        point_x.append(point[0])
        point_y.append(point[1])

        
        
plt.subplot(131)
plt.imshow(root_img)
plt.scatter(x=point_x, y=point_y, c='r', s=1)
plt.subplot(132)
plt.imshow(root_img)
plt.subplot(133)
plt.imshow(root_img)
plt.show()

IndexError: list index out of range